In [1]:
import pandas as pd
import plotly.express as px
from plotly.offline import plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
df = pd.read_excel('C:/Users/dimas/Jupyter/Stepik/payments_ru.xlsx')
infl = pd.read_excel('C:/Users/dimas/Jupyter/Stepik/inflation.xlsx')
infl['inflation'] = infl['inflation'] / 100
df.head()

,industry_old,industry,2000,2001,2002,2003,2004,2005,2006,2007,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Металлургическое производство и производство ...,Металлургическое производство,3854.9,5242.4,6285.2,7731.2,9196.8,10260.7,12001.5,14990.7,...,30437,33059,35310,49496.4,49982,53539.4,56928,63232.0,74947.7,85282.6
1,Производство кокса и нефтепродуктов,Производство кокса и нефти,4916.3,7012.4,9625.3,11879.0,13729.3,19397.1,22319.6,28565.0,...,75517,81605,88194,95956.7,87591,81685.2,82106,87214.4,97720.3,109541.0
2,Добыча полезных ископаемых,Добыча полезных ископаемых,5940.2,9099.2,11080.9,13912.4,16841.7,19726.9,23145.2,28107.5,...,58959,63695,69936,74474.1,83178,89343.7,95359,103473.5,118375.7,130825.6


In [3]:
df_melt = pd.melt(df, id_vars=['industry'], value_vars=df.columns[2:])
df_melt = df_melt.rename(columns={'variable':'year', 'value':'avg_payment'})
df_melt = df_melt.merge(infl, on='year', how='left')
df_melt['avg_payment_plus_infl'] = round(df_melt['avg_payment'] * (1+df_melt['inflation']), 1)
df_melt.head()

,industry,year,avg_payment,inflation,avg_payment_plus_infl
0,Металлургическое производство,2000,3854.9,0.2020,4633.6
1,Производство кокса и нефти,2000,4916.3,0.2020,5909.4
2,Добыча полезных ископаемых,2000,5940.2,0.2020,7140.1
3,Металлургическое производство,2001,5242.4,0.1858,6216.4
4,Производство кокса и нефти,2001,7012.4,0.1858,8315.3


In [4]:
df_melt_infl = pd.melt(df_melt, id_vars=['industry', 'year'], value_vars=['avg_payment', 'avg_payment_plus_infl'])
df_melt_infl['variable'] = df_melt_infl['variable'].replace({'avg_payment':'Средняя зарплата', 
                                                             'avg_payment_plus_infl':'Реальная зарплата'})

In [5]:
df_melt_infl.head()

,industry,year,variable,value
0,Металлургическое производство,2000,Средняя зарплата,3854.9
1,Производство кокса и нефти,2000,Средняя зарплата,4916.3
2,Добыча полезных ископаемых,2000,Средняя зарплата,5940.2
3,Металлургическое производство,2001,Средняя зарплата,5242.4
4,Производство кокса и нефти,2001,Средняя зарплата,7012.4


In [6]:
df_melt_sorted = df_melt.sort_values(by=['industry','year'])
df_melt_sorted = df_melt_sorted.set_index('year')
df_melt_sorted['lag'] = df_melt_sorted.groupby(['industry'])['avg_payment'].shift(1)
df_melt_sorted['payment_diff'] = df_melt_sorted['avg_payment'] - df_melt_sorted['lag']

df_melt_sorted.head()

,industry,avg_payment,inflation,avg_payment_plus_infl,lag,payment_diff
year,,,,,,
2000,Добыча полезных ископаемых,5940.2,0.2020,7140.1,NaN,NaN
2001,Добыча полезных ископаемых,9099.2,0.1858,10789.8,5940.2,3159.0
2002,Добыча полезных ископаемых,11080.9,0.1506,12749.7,9099.2,1981.7
2003,Добыча полезных ископаемых,13912.4,0.1199,15580.5,11080.9,2831.5
2004,Добыча полезных ископаемых,16841.7,0.1174,18818.9,13912.4,2929.3


In [7]:
corr = df_melt_sorted.groupby('industry')[['payment_diff', 'inflation']].corr()

In [8]:
corr[::2]

,,payment_diff,inflation
industry,,,
Добыча полезных ископаемых,payment_diff,1.0,-0.243469
Металлургическое производство,payment_diff,1.0,-0.281703
Производство кокса и нефти,payment_diff,1.0,0.185069


In [11]:
fig = px.line(df_melt_infl, x='year', y = 'value', color='variable', facet_col='industry', 
              title="Изменение фактических и реальных зарплат в РФ по годам",
              labels={
                     "value": "Зарплата, руб.",
                     "year": "Год",
                     "variable": "Тип зарплаты"   
                 })
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[0]))

fig.show() 

In [12]:
df_melt_sorted.head()

,industry,avg_payment,inflation,avg_payment_plus_infl,lag,payment_diff
year,,,,,,
2000,Добыча полезных ископаемых,5940.2,0.2020,7140.1,NaN,NaN
2001,Добыча полезных ископаемых,9099.2,0.1858,10789.8,5940.2,3159.0
2002,Добыча полезных ископаемых,11080.9,0.1506,12749.7,9099.2,1981.7
2003,Добыча полезных ископаемых,13912.4,0.1199,15580.5,11080.9,2831.5
2004,Добыча полезных ископаемых,16841.7,0.1174,18818.9,13912.4,2929.3


In [13]:
fig = px.line(df_melt_sorted, x=df_melt_sorted.index, y = 'payment_diff', color='industry',
              title="Динамика разницы зарплат в сравнении с предыдущим годом, руб",
              labels={
                     "payment_diff": "Разница , руб.",
                     "year": "Год",
                     "industry": "Направление"
                 })
fig.show()

fig = px.bar(infl, x='year', y='inflation',
             title="Изменение инфляции в РФ по годам, %",
             labels={
                     "inflation": "Инфляция",
                     "year": "Год"  
                 })
fig.show()

Выводы по пикам на графике (2008, 2015, 2022)

Добавить по 5 пункту - влияние инфляции на рост зп по сравнению с предыдущим годом